In [7]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors3D
from rdkit.Chem import rdChemReactions
from rdkit.Chem import Descriptors
from rdkit.Chem import Lipinski
from rdkit.Chem import rdChemReactions
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import Crippen
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw
from rdkit.Chem import SaltRemover
import pandas as pd
import csv
import freesasa
import numpy as np
import itertools

In [16]:
dir_name = '/Users/ztan/Documents/DEL data analysis/lib enumeration/lib8/'

df = pd.read_csv('Lib ID 8 (DOS-DEL-5) BB Structure.csv')
dfe = pd.read_csv('lib8_enum.csv')

#reductive aminations with all sorts of requirements on amine
rxn_redam = rdChemReactions.ReactionFromSmarts('[O:4]=[CH:2][c:3].[#7;H1,H2!$(NS);!$(NC=O);!$(NC=C);!$(NC=N):1]>>[c:3][CH2:2][N:1]')
rxn_cuaac = rdChemReactions.ReactionFromSmarts('[#6:3][C:1]#[CH:2].[C:4][N:5]=[N:6]=[N:7]>>[C:4][N:5]1[C:2]=[C:1]([#6:3])[N+0:7]=[N+0:6]1')


remover = Chem.SaltRemover.SaltRemover(defnData="[Cl,Br]")

structures = []
emw = []
fsp3 = []
nrb = []
slogp = []
libid = [8]
counter = 0


In [23]:
#real code here
structures = []
for lib in libid:
    cy1 = df[(df['master_lib_id'] == lib) & (df['cy_no'] == 1)]
    cy2 = df[(df['master_lib_id'] == lib) & (df['cy_no'] == 2)]
    cy3 = df[(df['master_lib_id'] == lib) & (df['cy_no'] == 3)]
    for i in range(len(cy1)):
        r1 = cy1.iloc[i]['bb_smiles']
        
        for j in range(len(cy2)):
            r2 = cy2.iloc[j]['bb_smiles']
            r2 = Chem.MolToSmiles(remover.StripMol(Chem.MolFromSmiles(r2)))
            products = rxn_cuaac.RunReactants((Chem.MolFromSmiles(r2),Chem.MolFromSmiles(r1)))
            p2 = Chem.MolToSmiles(products[0][0], isomericSmiles = True)
            
            for k in range(len(cy3)):
                r3 = cy3.iloc[k]['bb_smiles']
                r3 = Chem.MolToSmiles(remover.StripMol(Chem.MolFromSmiles(r3)))
                products = rxn_redam.RunReactants((Chem.MolFromSmiles(p2),Chem.MolFromSmiles(r3)))
                
                struct_out = Chem.MolToSmiles(products[0][0], isomericSmiles = True)
                structures.append(Chem.MolToSmiles(products[0][0], isomericSmiles = True))
                
                cpd_out = Chem.MolFromSmiles(struct_out)
                try:
                    emw.append(Descriptors.ExactMolWt(cpd_out))
                except:
                    emw.append(np.nan)
                try:
                    fsp3.append(Lipinski.FractionCSP3(cpd_out))
                except:
                    fsp3.append(np.nan)
                try:
                    nrb.append(Lipinski.NumRotatableBonds(cpd_out))
                except:
                    nrb.append(np.nan)
                try:
                    slogp.append(Crippen.MolLogP(cpd_out))
                except:
                    slogp.append(np.nan)
                    counter += 1
                

dfe['structures'] = structures
dfe.to_csv('lib8_enum.csv', index = False)
len(structures)
len(emw)

lib_id = 8
pd.DataFrame({'structures': structures}).to_csv(dir_name + 'meta/lib/enum_struct/lib' + str(lib_id).zfill(3) + '.csv', index = False)
pd.DataFrame({'value': emw}).to_csv(dir_name + 'meta/lib/enum_prop/lib' + str(lib_id).zfill(3) + '_emw.csv', index = False)
pd.DataFrame({'value': fsp3}).to_csv(dir_name + 'meta/lib/enum_prop/lib' + str(lib_id).zfill(3) + '_fsp3.csv', index = False)
pd.DataFrame({'value': nrb}).to_csv(dir_name + 'meta/lib/enum_prop/lib' + str(lib_id).zfill(3) + '_nrb.csv', index = False)
pd.DataFrame({'value': slogp}).to_csv(dir_name + 'meta/lib/enum_prop/lib' + str(lib_id).zfill(3) + '_slogp.csv', index = False)

[14:23:07] mapped atoms in the reactants were not mapped in the products.
  unmapped numbers are: 4 


In [34]:
agg_meta = pd.read_csv(dir_name + 'meta/an/agg_meta.csv')

In [37]:
# Generate meta cols
tags = [cy1['cy_id'], cy2['cy_id'], cy3['cy_id']]
all_combinations = list(itertools.product(*tags))
enum_df = pd.concat([
    pd.DataFrame({'lib_id': [lib_id for x in all_combinations]}),
    pd.DataFrame(all_combinations, columns=['cycle1', 'cycle2', 'cycle3'])
], axis = 1)

for i in range(len(agg_meta)):
    agg_id = agg_meta.iloc[i]['agg_id']
    col_names = ['cycle' + x for x in agg_meta.iloc[i]['cyc'].split(', ')]
    col_names.insert(0, 'lib_id')
    
    enum_out = enum_df[col_names].drop_duplicates()
    enum_out.to_csv(dir_name + 'meta/lib/meta_cols/lib' + str(lib_id).zfill(3) + '_' +
                    'agg' + str(agg_id).zfill(2) + '_meta_cols.csv', index = False)

0